In [47]:
import sys
import os

import pandas as pd

In [48]:
sys.path.append(os.path.abspath('../../src'))

from read_and_write_docs import read_xml, read_jsonl
from tokenize_and_score import load_model, compute_log_probs_with_median

In [49]:
known_text = read_xml("/Volumes/BCross/datasets/test_sms/adjusted/SMS_ENG_20110818.0003.conv.xml")
unknown_text = read_xml("/Volumes/BCross/datasets/test_sms/adjusted/SMS_ENG_20110901.0001.conv.xml")

In [81]:
known_text = read_jsonl("/Volumes/BCross/datasets/author_verification/training/Wiki/known_corpus_split/classicjupiter2_text_4.jsonl").loc[0, 'text']
unknown_text = read_jsonl("/Volumes/BCross/datasets/author_verification/training/Wiki/known_corpus_split/habap_text_3.jsonl").loc[0, 'text']

In [ ]:
{2: {('a', 'note'),
  ('drop', 'you'),
  ('i', 'just'),
  ('i', 'was'),
  ('i', 'will'),
  ('if', 'you'),
  ('in', 'the'),
  ('is', 'a'),
  ('is', 'also'),
  ('is', 'not'),
  ('just', 'wanted'),
  ('of', 'the'),
  ('on', 'the'),
  ('talk', 'page'),
  ('that', 'there'),
  ('the', 'article'),
  ('the', 'discussion'),
  ('the', 'page'),
  ('there', 'is'),
  ('to', 'be'),
  ('to', 'drop'),
  ('to', 'the'),
  ('wanted', 'to'),
  ('was', 'only'),
  ('you', 'a'),
  ('you', 'can')},
 3: {('drop', 'you', 'a'),
  ('just', 'wanted', 'to'),
  ('there', 'is', 'a'),
  ('to', 'drop', 'you'),
  ('wanted', 'to', 'drop'),
  ('you', 'a', 'note')},
 4: {('drop', 'you', 'a', 'note'),
  ('just', 'wanted', 'to', 'drop'),
  ('to', 'drop', 'you', 'a'),
  ('wanted', 'to', 'drop', 'you')},
 5: {('just', 'wanted', 'to', 'drop', 'you'),
  ('to', 'drop', 'you', 'a', 'note'),
  ('wanted', 'to', 'drop', 'you', 'a')},
 6: {('just', 'wanted', 'to', 'drop', 'you', 'a'),
  ('wanted', 'to', 'drop', 'you', 'a', 'note')},
 7: {('just', 'wanted', 'to', 'drop', 'you', 'a', 'note')}}

In [82]:
base_phrase = "bell me wen u"
base_phrase = "just wanted to drop you a note"

In [83]:
paraphrase_list = [
    "bel me wen u",
    "bel me wen ya",
    "bel me wen yu",
    "bel me whn u",
    "bell me wen ya",
    "bell me wen yu",
    "bell me whn u",
    "bell me when u",
    "bell me when you",
    "ring me wen u",
    "ring me wen ya",
    "ring me wen yu",
    "ring me whn u",
    "ring me when u",
    "ring me when you",
    "call me wen u",
    "call me wen ya",
    "call me wen yu",
    "call me whn u",
    "call me when u",
    "call me when you",
    "buzz me wen u",
    "buzz me wen ya",
    "buzz me wen yu",
    "buzz me whn u",
    "buzz me when u",
    "buzz me when you",
    "give me a bell wen u",
    "give me a ring wen u",
    "give me a call wen u",
    "gimme a bell wen u",
    "gimme a ring wen u",
    "gimme a call wen u",
    "hit me wen u",
    "hit me wen ya",
    "hit me wen yu",
    "hit me whn u",
    "hit me when u",
    "hit me when you",
    "phone me wen u",
    "phone me wen ya",
    "phone me wen yu",
    "phone me whn u",
    "phone me when u",
    "phone me when you",
    "holla wen u",
    "holla at me wen u",
    "holla when u",
    "holler wen u",
    "holler when u",
]

In [95]:
paraphrase_list = [
    "just wanted to drop a note to you",
    "just wanted to drop you a message",
    "just wanted to drop a message to you",
    "just wanted to drop you a line",
    "just wanted to drop a line to you",
    "just wanted to send you a note",
    "just wanted to send a note to you",
    "just wanted to send you a message",
    "just wanted to send a message to you",
    "just wanted to send you a line",
    "just wanted to send a line to you",
    "just wanted to leave you a note",
    "just wanted to leave a note for you",
    "just wanted to leave you a message",
    "just wanted to leave a message for you",
    "just wanted to give you a note",
    "just wanted to give a note to you",
    "just wanted to give you a message",
    "just wanted to give a message to you",
    "just wanted to pass you a note",
    "just wanted to pass a note to you",
    "just wanted to pass a message to you", 
    "just wanted to drop ya a note",
    "just wanted to drop a note to ya",
    "just wanted to send ya a note",
    "just wanted to send a note to ya",
    "just wanted to leave ya a note",
    "just wanted to leave a note for ya",
    "just wanted to give ya a note",
    "just wanted to give a note to ya",
    "just wanted to pass ya a note",
    "just wanted to pass a note to ya",
    "only wanted to drop you a note",
    "simply wanted to drop you a note",
    "only wanted to send you a note",
    "simply wanted to send you a note"
]


In [96]:
def keep_before_phrase(text: str, phrase: str, case_insensitive: bool = False) -> str:
    """
    Return everything in `text` before the first occurrence of `phrase`.
    If `phrase` isn’t found, returns the entire `text`.

    :param text:       The full string you want to trim.
    :param phrase:     The substring (phrase) you want to stop at.
    :param case_insensitive:  If True, match phrase ignoring case.
    :return:           The portion of `text` before `phrase`.
    """
    if case_insensitive:
        idx = text.lower().find(phrase.lower())
    else:
        idx = text.find(phrase)

    return text[:idx] if idx != -1 else text

In [97]:
base_known_text = keep_before_phrase(known_text, base_phrase)
base_unknown_text = keep_before_phrase(unknown_text, base_phrase)

In [ ]:
base_known_text

"International Surrealism was only recognized by the public and the historians and scholars between the years, 1924 to 1969.\nFrom the late 1960's to 2002's SURREALISM DESIRE UNBOUND Retrospective, the public acknowledges that International Surrealism is a retrospective movement in the arts and literature composed of notable figures who came and went, in and out of the International Surrealist Movement, like Aragon, Dali, Ernst, Matta, etc.\nBetween the years, 2003 to today, there is a surrealist presence online from artists and others that claim to be surrealist, and in fairness to the everyone that claims the surrealist label, its the public that is the final judge.\nWikipedia is an online encyclopedia, and it should not be used for self-promotion of non-notable groups that claim to be surrealists, just because they say so.\nThe Internet has literally hundreds of surreal and surrealist related material, art, etc, and we would open the floodgates for misinformation on this encyclopedi

In [98]:
tokenizer, model = load_model("/Volumes/BCross/models/Qwen 2.5/Qwen2.5-0.5B-Instruct")

In [99]:
from typing import List, Dict

def run_over_phrases(
    base_known: str,
    phrases: List[str],
    tokenizer,
    model
) -> Dict[str, Dict]:
    """
    For each phrase in `phrases`, prepend base_known, run compute_log_probs_with_median,
    and store tokens, log_probs, median_logprobs, and sum_log_probs.
    """
    results = {}
    total = len(phrases)
    for idx, phrase in enumerate(phrases, start=1):
        print(f"Scoring phrase {idx} out of {total}: {phrase}")
        text = base_known + phrase
        tokens, log_probs, median_logprobs = compute_log_probs_with_median(text, tokenizer, model)
        results[phrase] = {
            "text": text,
            "tokens":            tokens,
            "log_probs":         log_probs,
            "median_logprobs":   median_logprobs,
            "sum_log_probs":     sum(log_probs),
        }
    return results

In [100]:
# result = run_over_phrases(base_known_text, paraphrase_list, tokenizer, model)

In [101]:
import pandas as pd
from typing import List

def score_phrases(
    base_text: str,
    ref_phrase: str,
    paraphrases: List[str],
    tokenizer,
    model
) -> pd.DataFrame:
    """
    - Scores base_text alone → base_total and n_base_tokens
    - Then scores base_text + ref_phrase and each base_text + paraphrase,
      printing:
        • full sequence sum (`sum_before`)
        • phrase-only sum (`phrase_total`)
        • the phrase text
        • `difference`
    - Trims off the base_text tokens to isolate phrase-only tokens/log-probs.
    - Returns a DataFrame with columns:
        phrase_type, phrase, tokens, base_total, sum_before,
        log_probs, phrase_total, difference
    """
    # 1) score base_text alone
    print("Scoring base_text alone...")
    _, log_probs_base, _ = compute_log_probs_with_median(base_text, tokenizer, model)
    base_total    = sum(log_probs_base)
    n_base_tokens = len(log_probs_base)

    # prepare items
    all_items = [("reference", ref_phrase)] + [("paraphrase", p) for p in paraphrases]
    total = len(all_items)
    rows = []

    for idx, (ptype, phrase) in enumerate(all_items, start=1):
        print(f"Scoring {ptype} {idx}/{total}…")
        text = base_text + phrase
        tokens_all, log_probs_all, _ = compute_log_probs_with_median(text, tokenizer, model)

        sum_before       = sum(log_probs_all)
        # isolate phrase-only
        phrase_tokens    = tokens_all[n_base_tokens:]
        phrase_log_probs = log_probs_all[n_base_tokens:]
        phrase_total     = sum(phrase_log_probs)
        difference       = base_total - sum_before

        rows.append({
            "phrase_type":  ptype,
            "phrase":       phrase,
            "tokens":       phrase_tokens,
            "base_total":   base_total,
            "sum_before":   sum_before,
            "log_probs":    phrase_log_probs,
            "phrase_total": phrase_total,
            "difference":   difference
        })

    df = pd.DataFrame(rows, columns=[
        "phrase_type",
        "phrase",
        "tokens",
        "base_total",
        "sum_before",
        "log_probs",
        "phrase_total",
        "difference"
    ])
    return df


In [122]:
import pandas as pd
from typing import List

import torch
import pandas as pd
from typing import List

def compute_log_probs_with_median(text: str, tokenizer, model):
    """
    For each token (including the first), returns:
      - tokens: list of tokenizer.convert_ids_to_tokens
      - log_probs: list of log-probs for each token
      - median_logprobs: median log-prob of the distribution at each step
    """
    inputs = tokenizer(text, return_tensors="pt")
    input_ids = inputs["input_ids"]         # shape [1, seq_len]
    # --- ALIGN TOKENS CORRECTLY HERE ---
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

    with torch.no_grad():
        outputs = model(input_ids)
    logits = outputs.logits                 # [1, seq_len, vocab_size]

    log_probs = []
    median_logprobs = []
    # for each position i, look at logits from i-1 (or the BOS for i=0)
    for i in range(input_ids.size(1)):
        prev_idx = 0 if i == 0 else i - 1
        dist = torch.log_softmax(logits[0, prev_idx], dim=-1)
        log_prob = dist[input_ids[0, i]].item()
        median_lp = float(dist.median().item())
        log_probs.append(log_prob)
        median_logprobs.append(median_lp)

    return tokens, log_probs, median_logprobs

def score_phrases(
    base_text: str,
    ref_phrase: str,
    paraphrases: List[str],
    tokenizer,
    model
) -> pd.DataFrame:
    """
    1) Score base_text alone → base_total
    2) For each phrase (reference + paraphrases):
         a) Get its token count by scoring phrase alone
         b) Score base_text + phrase → full tokens & log_probs
         c) sum_before = sum(full log_probs)
         d) phrase_tokens    = last n_phrase tokens of full tokens
         e) phrase_log_probs = last n_phrase values of full log_probs
         f) phrase_total     = sum(phrase_log_probs)
         g) difference       = base_total - sum_before
         h) APPEND row
    3) Return DataFrame with columns:
       phrase_type, phrase, tokens, base_total, sum_before,
       log_probs, phrase_total, difference
    """
    # 1) score base_text
    print("→ Scoring base_text alone…")
    _, log_probs_base, _ = compute_log_probs_with_median(base_text.strip(), tokenizer, model)
    base_total = sum(log_probs_base)
    print(f"   base_total = {base_total:.4f}\n")

    items = [("reference", ref_phrase)] + [("paraphrase", p) for p in paraphrases]
    rows = []

    for idx, (ptype, phrase) in enumerate(items, start=1):
        print(f"→ [{idx}/{len(items)}] Processing {ptype}…")

        # a) phrase alone → get token count
        tokens_phrase, log_probs_phrase, _ = compute_log_probs_with_median(phrase, tokenizer, model)
        n_phrase_tokens = len(tokens_phrase)
        # b) full sequence
        full_text = base_text + phrase
        tokens_full, log_probs_full, _ = compute_log_probs_with_median(full_text, tokenizer, model)
        # c) full sum
        sum_before = sum(log_probs_full)
        # d/e) slice last n_phrase_tokens
        phrase_tokens    = tokens_full[-n_phrase_tokens:]
        phrase_log_probs = log_probs_full[-n_phrase_tokens:]
        # f/g) compute sums
        phrase_total = sum(phrase_log_probs)
        difference   = base_total - sum_before
        # h) collect
        rows.append({
            "phrase_type":  ptype,
            "phrase":       phrase,
            "tokens":       phrase_tokens,
            "sum_log_probs_base":   base_total,
            "sum_log_probs_inc_phrase":   sum_before,
            "difference":   difference,
            "phrase_log_probs":    phrase_log_probs,
            "sum_log_probs_phrase": phrase_total,
        })

    return pd.DataFrame(rows, columns=[
        "phrase_type", "phrase", "tokens",
        "sum_log_probs_base", "sum_log_probs_inc_phrase",
        "difference", "phrase_log_probs", "sum_log_probs_phrase",
    ])


In [123]:
# import torch
# import pandas as pd
# from typing import List

# def compute_log_probs_with_median(text: str, tokenizer, model):
#     """
#     For each token (including the first), returns:
#       - tokens: list of tokenizer.convert_ids_to_tokens
#       - log_probs: list of log-probs for each token
#       - median_logprobs: median log-prob of the distribution at each step
#     """
#     inputs = tokenizer(text, return_tensors="pt")
#     input_ids = inputs["input_ids"]         # shape [1, seq_len]
#     # --- ALIGN TOKENS CORRECTLY HERE ---
#     tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

#     with torch.no_grad():
#         outputs = model(input_ids)
#     logits = outputs.logits                 # [1, seq_len, vocab_size]

#     log_probs = []
#     median_logprobs = []
#     # for each position i, look at logits from i-1 (or the BOS for i=0)
#     for i in range(input_ids.size(1)):
#         prev_idx = 0 if i == 0 else i - 1
#         dist = torch.log_softmax(logits[0, prev_idx], dim=-1)
#         log_prob = dist[input_ids[0, i]].item()
#         median_lp = float(dist.median().item())
#         log_probs.append(log_prob)
#         median_logprobs.append(median_lp)

#     return tokens, log_probs, median_logprobs


# def score_phrases(
#     base_text: str,
#     ref_phrase: str,
#     paraphrases: List[str],
#     tokenizer,
#     model
# ) -> pd.DataFrame:
#     """
#     1) Score base_text → base_total & n_base_tokens
#     2) For ref_phrase + each paraphrase:
#         • Score base_text+phrase → tokens_all, log_probs_all
#         • sum_before = sum(log_probs_all)
#         • phrase_tokens = tokens_all[n_base_tokens:]
#         • phrase_log_probs = log_probs_all[n_base_tokens:]
#         • phrase_total = sum(phrase_log_probs)
#         • difference   = base_total - sum_before
#         • PRINT sum_before, phrase_total, phrase, difference
#         • APPEND to rows
#     3) Return DataFrame with columns:
#        phrase_type, phrase, tokens, base_total, sum_before,
#        log_probs, phrase_total, difference
#     """
#     # 1) score base_text alone
#     print("→ Scoring base_text alone…")
#     tokens_base, log_probs_base, _ = compute_log_probs_with_median(base_text, tokenizer, model)
#     base_total    = sum(log_probs_base)
#     n_base_tokens = len(tokens_base)
#     print(f"   base_total = {base_total:.4f} over {n_base_tokens} tokens\n")

#     all_items = [("reference", ref_phrase)] + [("paraphrase", p) for p in paraphrases]
#     rows = []
#     total = len(all_items)

#     for idx, (ptype, phrase) in enumerate(all_items, start=1):
#         print(f"→ [{idx}/{total}] Scoring {ptype}…")
#         text = base_text + phrase
#         tokens_all, log_probs_all, _ = compute_log_probs_with_median(text, tokenizer, model)

#         sum_before       = sum(log_probs_all)
#         # **NOW** this slice actually lines up
#         phrase_tokens    = tokens_all[n_base_tokens-1:]
#         phrase_log_probs = log_probs_all[n_base_tokens-1:]
#         phrase_total     = sum(phrase_log_probs)
#         difference       = base_total - sum_before

#         rows.append({
#             "phrase_type":  ptype,
#             "phrase":       phrase,
#             "tokens":       phrase_tokens,
#             "base_total":   base_total,
#             "sum_before":   sum_before,
#             "log_probs":    phrase_log_probs,
#             "phrase_total": phrase_total,
#             "difference":   difference
#         })

#     df = pd.DataFrame(rows, columns=[
#         "phrase_type", "phrase", "tokens",
#         "base_total", "sum_before", "log_probs",
#         "phrase_total", "difference"
#     ])
#     return df


In [124]:
results_known = score_phrases(base_known_text, base_phrase, paraphrase_list, tokenizer, model)

→ Scoring base_text alone…
   base_total = -1583.6236

→ [1/37] Processing reference…
→ [2/37] Processing paraphrase…
→ [3/37] Processing paraphrase…
→ [4/37] Processing paraphrase…
→ [5/37] Processing paraphrase…
→ [6/37] Processing paraphrase…
→ [7/37] Processing paraphrase…
→ [8/37] Processing paraphrase…
→ [9/37] Processing paraphrase…
→ [10/37] Processing paraphrase…
→ [11/37] Processing paraphrase…
→ [12/37] Processing paraphrase…
→ [13/37] Processing paraphrase…
→ [14/37] Processing paraphrase…
→ [15/37] Processing paraphrase…
→ [16/37] Processing paraphrase…
→ [17/37] Processing paraphrase…
→ [18/37] Processing paraphrase…
→ [19/37] Processing paraphrase…
→ [20/37] Processing paraphrase…
→ [21/37] Processing paraphrase…
→ [22/37] Processing paraphrase…
→ [23/37] Processing paraphrase…
→ [24/37] Processing paraphrase…
→ [25/37] Processing paraphrase…
→ [26/37] Processing paraphrase…
→ [27/37] Processing paraphrase…
→ [28/37] Processing paraphrase…
→ [29/37] Processing paraphrase

In [141]:
results_known.sort_values(by='difference', inplace=True)
results_known.head(2)

,phrase_type,phrase,tokens,sum_log_probs_base,sum_log_probs_inc_phrase,difference,phrase_log_probs,sum_log_probs_phrase
8,paraphrase,just wanted to send you a message,"[Ġjust, Ġwanted, Ġto, Ġsend, Ġyou, Ġa, Ġmessage]",-1583.623627,-1600.023247,16.399620,"[-5.656452178955078, -2.6228346824645996, -0.0...",-16.399620
4,paraphrase,just wanted to drop you a line,"[Ġjust, Ġwanted, Ġto, Ġdrop, Ġyou, Ġa, Ġline]",-1583.623627,-1601.297384,17.673757,"[-5.656452178955078, -2.6228346824645996, -0.0...",-17.673757


In [126]:
results_known.to_clipboard()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [130]:
import math

def logsumexp(xs):
    m = max(xs)
    return m + math.log(sum(math.exp(x - m) for x in xs))

def p_ref_among_options(logprob_lists, priors=None, ref_index=0):
    L = [sum(lp) for lp in logprob_lists]
    if priors is not None:
        L = [Li + math.log(pi) for Li, pi in zip(L, priors)]
    return math.exp(L[ref_index] - logsumexp(L))

In [136]:
import math

def logsumexp(xs):
    m = max(xs)
    return m + math.log(sum(math.exp(x - m) for x in xs))

def ref_metrics(token_logprob_lists, priors=None, ref_index=0):
    # sequence log-likelihoods
    L = [sum(lst) for lst in token_logprob_lists]
    if priors is not None:
        L = [Li + math.log(pi) for Li, pi in zip(L, priors)]

    logZ = logsumexp(L)
    p_ref = math.exp(L[ref_index] - logZ)

    others = [L[i] for i in range(len(L)) if i != ref_index]
    log_den_others = logsumexp(others)
    llr_ref_vs_rest = L[ref_index] - log_den_others     # = logit(p_ref)
    odds_ref = math.exp(llr_ref_vs_rest)

    pmf = [math.exp(Li - logZ) for Li in L]            # full normalized PMF

    return {
        "p_ref": p_ref,
        "pmf": pmf,
        "log_den": logZ,            # numerically stable
        "den": math.exp(logZ),      # may overflow/underflow
        "llr_ref_vs_rest": llr_ref_vs_rest,
        "odds_ref": odds_ref
    }


In [133]:
results_known.sort_values(by='phrase_type', ascending=False, inplace=True)

p_ref_among_options(results_known['phrase_log_probs'].tolist())

0.12267416407896513

In [137]:
ref_metrics(results_known['phrase_log_probs'].tolist())

{'p_ref': 0.12267416407896513,
 'pmf': [0.12267416407896513,
  0.0017798566555540494,
  0.0125798948455079,
  0.12657870523569342,
  0.06738147396391961,
  0.018400070914360057,
  0.014097329110571225,
  0.013937170982132843,
  0.012674830969522089,
  0.006810618166642372,
  0.0017889109618277436,
  0.0062875900431353824,
  0.0007158860450894309,
  0.004117177263267585,
  0.000434469539497419,
  0.0028527199424338507,
  0.0026874769234762367,
  0.12691055836512496,
  0.000895849705025534,
  0.4537831787918548,
  7.693809488150951e-07,
  2.992009824435929e-05,
  0.000626254290402676,
  0.00029521942142091335,
  0.00023608819831597998,
  0.00015680944972811312,
  0.00012433316791233835,
  0.00010197514528528433,
  3.969938516975028e-05,
  2.8915014733866484e-05,
  0.0009289088465866804,
  2.6158133359340097e-05,
  8.432866211287504e-06,
  3.666536183207843e-06,
  3.538822827199206e-06,
  1.1761084605139176e-06,
  2.0263060763071356e-07],
 'log_den': -15.609484244127495,
 'den': 1.6629803

In [142]:
sum(results_known['sum_log_probs_phrase'].tolist())

-849.7581649534404

In [127]:
results_unknown = score_phrases(base_unknown_text, base_phrase, paraphrase_list, tokenizer, model)

→ Scoring base_text alone…
   base_total = -2347.6514

→ [1/37] Processing reference…
→ [2/37] Processing paraphrase…
→ [3/37] Processing paraphrase…
→ [4/37] Processing paraphrase…
→ [5/37] Processing paraphrase…
→ [6/37] Processing paraphrase…
→ [7/37] Processing paraphrase…
→ [8/37] Processing paraphrase…
→ [9/37] Processing paraphrase…
→ [10/37] Processing paraphrase…
→ [11/37] Processing paraphrase…
→ [12/37] Processing paraphrase…
→ [13/37] Processing paraphrase…
→ [14/37] Processing paraphrase…
→ [15/37] Processing paraphrase…
→ [16/37] Processing paraphrase…
→ [17/37] Processing paraphrase…
→ [18/37] Processing paraphrase…
→ [19/37] Processing paraphrase…
→ [20/37] Processing paraphrase…
→ [21/37] Processing paraphrase…
→ [22/37] Processing paraphrase…
→ [23/37] Processing paraphrase…
→ [24/37] Processing paraphrase…
→ [25/37] Processing paraphrase…
→ [26/37] Processing paraphrase…
→ [27/37] Processing paraphrase…
→ [28/37] Processing paraphrase…
→ [29/37] Processing paraphrase

In [150]:
results_unknown.sort_values(by='difference', inplace=True)
results_unknown.head(5)

,phrase_type,phrase,tokens,sum_log_probs_base,sum_log_probs_inc_phrase,difference,phrase_log_probs,sum_log_probs_phrase
0,reference,just wanted to drop you a note,"[.just, Ġwanted, Ġto, Ġdrop, Ġyou, Ġa, Ġnote]",-2347.651417,-2366.505260,18.853843,"[-18.542102813720703, -1.2133077383041382, -0....",-21.993745
6,paraphrase,just wanted to send you a note,"[.just, Ġwanted, Ġto, Ġsend, Ġyou, Ġa, Ġnote]",-2347.651417,-2370.240882,22.589465,"[-18.542102813720703, -1.2133077383041382, -0....",-25.729366
33,paraphrase,only wanted to drop you a note,"[.only, Ġwanted, Ġto, Ġdrop, Ġyou, Ġa, Ġnote]",-2347.651417,-2370.678002,23.026585,"[-20.74664306640625, -2.797487258911133, -0.02...",-26.166487
34,paraphrase,simply wanted to drop you a note,"[.sim, ply, Ġwanted, Ġto, Ġdrop, Ġyou, Ġa, Ġnote]",-2347.651417,-2370.883709,23.232292,"[-19.088476181030273, -0.2531338334083557, -3....",-26.372194
2,paraphrase,just wanted to drop you a message,"[.just, Ġwanted, Ġto, Ġdrop, Ġyou, Ġa, Ġmessage]",-2347.651417,-2371.118758,23.467341,"[-18.542102813720703, -1.2133077383041382, -0....",-26.607242


In [129]:
results_unknown.to_clipboard()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [135]:
results_unknown.sort_values(by='phrase_type', ascending=False, inplace=True)

p_ref_among_options(results_unknown['phrase_log_probs'].tolist())

0.9140128938128587

In [144]:
unk_metrics = ref_metrics(results_unknown['phrase_log_probs'].tolist())

In [147]:
unk_metrics

{'p_ref': 0.9140128938128587,
 'pmf': [0.9140128938128587,
  0.00016217135470962,
  4.784973369683106e-05,
  3.571054297152586e-05,
  2.1443270550867535e-05,
  1.2708328473742358e-05,
  7.457871773571731e-06,
  5.912474607660524e-06,
  3.987683067853873e-06,
  2.4226698073304474e-06,
  1.8995599646531103e-06,
  1.8479749160098645e-06,
  1.020550925898058e-06,
  7.595025255933304e-07,
  3.6509056688697477e-07,
  3.408860488113954e-07,
  6.500583407921985e-08,
  0.0001244046886345057,
  0.00017975772915759886,
  0.02180682103856921,
  0.00018677745182850718,
  0.014084886187533164,
  0.011466106350459549,
  0.009064328332442115,
  0.00669993353248597,
  0.00638473242985564,
  0.005240339306690617,
  0.0032569309362411336,
  0.0019164574733504629,
  0.0014093162653898397,
  0.001211408831988399,
  0.0006841709377676138,
  0.0006740666049030073,
  0.0005588811416520615,
  0.000384417909883419,
  0.0003473590114705051,
  4.7526398237230654e-08],
 'log_den': -21.903833932897783,
 'den': 3.07

In [146]:
math.exp(-21.993745)

2.806970898748768e-10

In [155]:
sum(math.exp(lp) for lp in results_unknown['sum_log_probs_phrase'])

3.0710422436889754e-10

In [156]:
math.exp(-21.993745)

2.806970898748768e-10

In [157]:
2.806970898748768e-10 / 3.0710422436889754e-10

0.9140124674341824

In [159]:
math.log(0.12267416407896513) - math.log(0.9140124674341824)

-2.008312444037906

In [ ]:
ref = compute_log_probs_with_median(base_phrase, tokenizer, model)

(['just', 'Ġwanted', 'Ġto', 'Ġdrop', 'Ġyou', 'Ġa', 'Ġnote'],
 [-13.547368049621582,
  -11.230600357055664,
  -0.07694364339113235,
  -5.435577392578125,
  -3.012918472290039,
  -0.5511003136634827,
  -2.96950101852417],
 [-16.516681671142578,
  -16.516681671142578,
  -22.0695858001709,
  -21.389036178588867,
  -20.315900802612305,
  -20.781362533569336,
  -21.7308349609375])

In [162]:
def score_phrases_no_context(
    ref_phrase: str,
    paraphrases: List[str],
    tokenizer,
    model
) -> pd.DataFrame:
    """
    1) Score base_text alone → base_total
    2) For each phrase (reference + paraphrases):
         a) Get its token count by scoring phrase alone
         b) Score base_text + phrase → full tokens & log_probs
         c) sum_before = sum(full log_probs)
         d) phrase_tokens    = last n_phrase tokens of full tokens
         e) phrase_log_probs = last n_phrase values of full log_probs
         f) phrase_total     = sum(phrase_log_probs)
         g) difference       = base_total - sum_before
         h) APPEND row
    3) Return DataFrame with columns:
       phrase_type, phrase, tokens, base_total, sum_before,
       log_probs, phrase_total, difference
    """
    # 1) score base_text
    items = [("reference", ref_phrase)] + [("paraphrase", p) for p in paraphrases]
    rows = []

    for idx, (ptype, phrase) in enumerate(items, start=1):
        print(f"→ [{idx}/{len(items)}] Processing {ptype}…")

        # a) phrase alone → get token count
        tokens_phrase, log_probs_phrase, _ = compute_log_probs_with_median(phrase, tokenizer, model)
        # b) compute sum
        phrase_total = sum(log_probs_phrase)
        # h) collect
        rows.append({
            "phrase_type":  ptype,
            "phrase":       phrase,
            "tokens":       tokens_phrase,
            "log_probs":    log_probs_phrase,
            "sum_log_probs": phrase_total,
        })

    return pd.DataFrame(rows, columns=[
        "phrase_type", "phrase", "tokens",
        "log_probs", "sum_log_probs",
    ])

In [163]:
phrase_no_context_results = score_phrases_no_context(base_phrase, paraphrase_list, tokenizer, model)

→ [1/37] Processing reference…
→ [2/37] Processing paraphrase…
→ [3/37] Processing paraphrase…
→ [4/37] Processing paraphrase…
→ [5/37] Processing paraphrase…
→ [6/37] Processing paraphrase…
→ [7/37] Processing paraphrase…
→ [8/37] Processing paraphrase…
→ [9/37] Processing paraphrase…
→ [10/37] Processing paraphrase…
→ [11/37] Processing paraphrase…
→ [12/37] Processing paraphrase…
→ [13/37] Processing paraphrase…
→ [14/37] Processing paraphrase…
→ [15/37] Processing paraphrase…
→ [16/37] Processing paraphrase…
→ [17/37] Processing paraphrase…
→ [18/37] Processing paraphrase…
→ [19/37] Processing paraphrase…
→ [20/37] Processing paraphrase…
→ [21/37] Processing paraphrase…
→ [22/37] Processing paraphrase…
→ [23/37] Processing paraphrase…
→ [24/37] Processing paraphrase…
→ [25/37] Processing paraphrase…
→ [26/37] Processing paraphrase…
→ [27/37] Processing paraphrase…
→ [28/37] Processing paraphrase…
→ [29/37] Processing paraphrase…
→ [30/37] Processing paraphrase…
→ [31/37] Processing

In [164]:
phrase_no_context_results

,phrase_type,phrase,tokens,log_probs,sum_log_probs
0,reference,just wanted to drop you a note,"[just, Ġwanted, Ġto, Ġdrop, Ġyou, Ġa, Ġnote]","[-13.547368049621582, -11.230600357055664, -0....",-36.824009
1,paraphrase,just wanted to drop a note to you,"[just, Ġwanted, Ġto, Ġdrop, Ġa, Ġnote, Ġto, Ġyou]","[-13.547368049621582, -11.230600357055664, -0....",-35.761074
2,paraphrase,just wanted to drop you a message,"[just, Ġwanted, Ġto, Ġdrop, Ġyou, Ġa, Ġmessage]","[-13.547368049621582, -11.230600357055664, -0....",-39.778634
3,paraphrase,just wanted to drop a message to you,"[just, Ġwanted, Ġto, Ġdrop, Ġa, Ġmessage, Ġto,...","[-13.547368049621582, -11.230600357055664, -0....",-37.104399
4,paraphrase,just wanted to drop you a line,"[just, Ġwanted, Ġto, Ġdrop, Ġyou, Ġa, Ġline]","[-13.547368049621582, -11.230600357055664, -0....",-34.001307
5,paraphrase,just wanted to drop a line to you,"[just, Ġwanted, Ġto, Ġdrop, Ġa, Ġline, Ġto, Ġyou]","[-13.547368049621582, -11.230600357055664, -0....",-38.315746
6,paraphrase,just wanted to send you a note,"[just, Ġwanted, Ġto, Ġsend, Ġyou, Ġa, Ġnote]","[-13.547368049621582, -11.230600357055664, -0....",-36.229169
7,paraphrase,just wanted to send a note to you,"[just, Ġwanted, Ġto, Ġsend, Ġa, Ġnote, Ġto, Ġyou]","[-13.547368049621582, -11.230600357055664, -0....",-38.441291
8,paraphrase,just wanted to send you a message,"[just, Ġwanted, Ġto, Ġsend, Ġyou, Ġa, Ġmessage]","[-13.547368049621582, -11.230600357055664, -0....",-34.957332
9,paraphrase,just wanted to send a message to you,"[just, Ġwanted, Ġto, Ġsend, Ġa, Ġmessage, Ġto,...","[-13.547368049621582, -11.230600357055664, -0....",-36.873977


In [165]:
phrase_no_context_results.to_clipboard()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [166]:
ref_metrics(phrase_no_context_results['log_probs'].tolist())

{'p_ref': 0.02942703608435714,
 'pmf': [0.02942703608435714,
  0.08518705975877248,
  0.00153309522815474,
  0.02223181571976897,
  0.49502898376941395,
  0.006620540822840208,
  0.05334360039450061,
  0.005839424315377672,
  0.19029799722300392,
  0.0279927686630204,
  0.00036593318764960883,
  4.4279190953408884e-07,
  0.016561591261916014,
  0.012231002196408955,
  0.015443991910654193,
  0.015744169946309096,
  0.01598870792759976,
  0.0019328462398086792,
  0.00202757444268726,
  0.00030843112278552016,
  0.0005150109174092671,
  0.0007774717419987056,
  0.00018063978406505414,
  0.00011064914097456609,
  0.00016519444097009707,
  2.539807869046445e-05,
  1.2079753541652437e-05,
  3.586496373663749e-05,
  4.6547386795370134e-05,
  9.675178073893586e-06,
  1.0786488684721038e-05,
  1.4202459703150708e-07,
  1.32861536726391e-06,
  1.0233554992817302e-10,
  5.5139786685459756e-14,
  2.1847368120972857e-06,
  1.3638956227979283e-08],
 'log_den': -33.29816781462455,
 'den': 3.45771712